In [43]:
import copy as cp
import math as mt
import numpy as np
import pandas as pd

def rmat(nr,nc): return(np.matrix(np.random.rand(nr,nc)))

def hey(s):
    from scipy.sparse import triu
    return np.matrix(triu(rmat(s,s)).toarray())
    

class parameters:
    def __init__(self,size):
        self.size = size
        self.mu = np.matrix(np.zeros((size,1)))
        self.F = np.matrix(np.zeros((size,size)))
        self.Q = np.matrix(np.zeros((size,size)))
        self.R = np.matrix(np.zeros((1,1)))
        self.beta00 = np.matrix(np.zeros((size,1)))
        self.P00 = np.matrix(np.zeros((size,size)))
        self.parnames = list(self.__dict__.keys())
        self.parnames.remove('size')
        self.parnames.sort()
        
    def pack(self):
        p = np.matrix(np.zeros((0,1)))
        for aname in parnames:
            mtrx = getattr(ibm_model.parameters,aname)
            newmtrx = np.reshape(mtrx,(-1,1),order='F')
            p = np.concatenate((p,newmtrx),axis=0)
        return(p)
    
    def unpack(self,p):
        pos = 0
        for aname in self.parnames:
            mtrx = getattr(self,aname)
            nr,nc = np.shape(mtrx)
            newpos = pos + nr*nc
            subp = np.matrix(p[pos:newpos])
            mtrx = np.reshape(subp,(nr,nc),order='F')
            setattr(self,aname,mtrx)
            pos = cp.copy(newpos)
            
        
    def randomize(self):
        s = self.size
        self.mu = rmat(s,1)
        self.F = rmat(s,s)
        Q = 上三角(s)
        self.Q = Q.transpose()*Q
        self.R = rmat(1,1)
        self.beta00 = rmat(s,1)
        P = 上三角(s)
        self.P00 = P.transpose()*P
        
        
class thedata:
    def __init__(self,size):
        self.y = np.matrix(np.zeros((1,1)))
        self.x = np.matrix(np.zeros((1,size)))
        
class predictions:
    def __init__(self,size):
        self.beta = np.matrix(np.zeros((size,1)))
        self.P = np.matrix(np.zeros((size,size)))
        self.eta = np.matrix(np.zeros((1,1)))
        self.f = np.matrix(np.zeros((1,1)))
        
class updating:
    def __init__(self,size):
        self.beta = np.matrix(np.zeros((size,1)))
        self.P = np.matrix(np.zeros((size,size)))
        self.K = np.matrix(np.zeros((size,1)))
        
class an_obs:
    def __init__(self,t,fltr):
        self.t = t
        self.fltr = fltr
        size = self.fltr.size
        self.data = thedata(size)
        self.prediction = predictions(size)
        self.updating = updating(size)
    def predict(self):
        if self.t == 0: 
            b00 = self.fltr.parameters.beta00
            P00 = self.fltr.parameters.P00
        else: 
            b00 = self.fltr.obs[self.t-1].updating.beta
            P00 = self.fltr.obs[self.t-1].updating.P
        F = self.fltr.parameters.F
        x = self.data.x
        self.prediction.beta = self.fltr.parameters.mu+F*b00
        self.prediction.P = F*P00*F.transpose()+self.fltr.parameters.Q
        eta = self.data.y - x*self.prediction.beta
        f = x*self.prediction.P*x.transpose()+self.fltr.parameters.R
        self.prediction.eta = eta
        self.prediction.f = f
        self.ll = mt.log(2*mt.pi*f) +eta.transpose()*np.linalg.inv(f)*eta
    def update(self):
        P = self.prediction.P
        x = self.data.x
        K = P*x.transpose()*np.linalg.inv(self.prediction.f)
        self.updating.K = K
        self.updating.beta = self.prediction.beta + K*self.prediction.eta
        self.updating.P = P - K*x*P

class a_filter:
    def __init__(self,thefile):
        data = pd.read_csv(thefile)
        nr,nc = np.shape(data)
        nx = nc-1
        self.intercept = False
        response = input ('Please enter "NO" if you do not want an intercept:')
        if (response != "NO"):
            nx = nx+1
            self.intercept=True
        self.size = nx
        self.T = nr
        self.parameters = parameters(self.size)
        self.obs =[]
        for t in range(self.T):
            newobs = an_obs(t,self)
            x = np.matrix(data.iloc[t,1:])
            y = np.matrix(data.iloc[t,0])
            if self.intercept:
                x = np.concatenate((np.matrix(1),x),axis=1)
            newobs.data.y = y
            newobs.data.x = x
            self.obs = self.obs + [newobs]
            
    def run(self):
        self.ll = 0
        for t in range(self.T):
            self.obs[t].predict()
            self.obs[t].update()
            self.ll = self.ll + self.obs[t].ll
        self.ll = -0.5*self.ll
            
            

In [44]:
ibm_model = a_filter('IBM.csv')
ibm_model.parameters.randomize()
p = ibm_model.parameters.pack()

Please enter "NO" if you do not want an intercept:


In [48]:
newparameters=parameters(4)

In [49]:
newparameters.__dict__

{'size': 4, 'mu': matrix([[0.],
         [0.],
         [0.],
         [0.]]), 'F': matrix([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]), 'Q': matrix([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]), 'R': matrix([[0.]]), 'beta00': matrix([[0.],
         [0.],
         [0.],
         [0.]]), 'P00': matrix([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]), 'parnames': ['F',
  'P00',
  'Q',
  'R',
  'beta00',
  'mu']}

In [50]:
newparameters.unpack(p)

In [51]:
newparameters.F

matrix([[0.66685526, 0.24404427, 0.68145512, 0.28858908],
        [0.09828899, 0.92989951, 0.12507959, 0.20320343],
        [0.38116914, 0.64128889, 0.8430657 , 0.42192508],
        [0.06297519, 0.52505803, 0.16832111, 0.37130485]])

In [52]:
ibm_model.parameters.F

matrix([[0.66685526, 0.24404427, 0.68145512, 0.28858908],
        [0.09828899, 0.92989951, 0.12507959, 0.20320343],
        [0.38116914, 0.64128889, 0.8430657 , 0.42192508],
        [0.06297519, 0.52505803, 0.16832111, 0.37130485]])

In [53]:
newparameters.Q

matrix([[0.10490709, 0.00486151, 0.2378881 , 0.05658843],
        [0.00486151, 0.01818698, 0.04360193, 0.10397398],
        [0.2378881 , 0.04360193, 1.57470871, 0.57665979],
        [0.05658843, 0.10397398, 0.57665979, 0.77255463]])

In [54]:
ibm_model.parameters.Q

matrix([[0.10490709, 0.00486151, 0.2378881 , 0.05658843],
        [0.00486151, 0.01818698, 0.04360193, 0.10397398],
        [0.2378881 , 0.04360193, 1.57470871, 0.57665979],
        [0.05658843, 0.10397398, 0.57665979, 0.77255463]])

In [34]:
ibm_model = a_filter('IBM.csv')

Please enter "NO" if you do not want an intercept:Yes


In [35]:
ibm_model.obs[7].data.x

matrix([[ 1.    , -0.0284, -0.0083, -0.0323]])

In [36]:
ibm_model.parameters.randomize()

In [37]:
ibm_model.parameters.pack()

matrix([[0.50825321],
        [0.12328829],
        [0.94122424],
        [0.29218026],
        [0.22146111],
        [0.38031799],
        [0.23362197],
        [0.02514464],
        [0.37920822],
        [0.05743224],
        [0.18457682],
        [0.15231578],
        [0.86789654],
        [0.72934411],
        [0.19310546],
        [0.85493608],
        [0.14990342],
        [0.12493194],
        [0.24908696],
        [0.14218803],
        [0.12493194],
        [0.25831855],
        [0.23370961],
        [0.43741362],
        [0.24908696],
        [0.23370961],
        [1.28114607],
        [0.94117558],
        [0.14218803],
        [0.43741362],
        [0.94117558],
        [2.2806519 ],
        [0.17418709],
        [0.01972034],
        [0.04452922],
        [0.41329856],
        [0.01972034],
        [0.30309784],
        [0.24314629],
        [0.45958758],
        [0.04452922],
        [0.24314629],
        [0.28405613],
        [0.49725836],
        [0.41329856],
        [0

In [64]:
ibm_model.obs[7].prediction.f

matrix([[0.]])

In [6]:
ibm_model.parameters.randomize()

In [7]:
ibm_model.run()

In [8]:
ibm_model.ll

matrix([[-630.82015911]])

In [9]:
ibm_model.parameters.__dict__.keys()

dict_keys(['size', 'mu', 'F', 'Q', 'R', 'beta00', 'P00'])

In [10]:
for akey in list(ibm_model.parameters.__dict__.keys()):
    print ('this is the key:'+akey)

this is the key:size
this is the key:mu
this is the key:F
this is the key:Q
this is the key:R
this is the key:beta00
this is the key:P00


In [11]:
F = ibm_model.parameters.F

In [12]:
np.shape(F)

(4, 4)

In [14]:
np.reshape(F,(16,1),order='F')

matrix([[0.63218146],
        [0.85628559],
        [0.32981306],
        [0.98939182],
        [0.69127342],
        [0.99533619],
        [0.46227985],
        [0.10466464],
        [0.18292136],
        [0.05287674],
        [0.71755215],
        [0.75474602],
        [0.44698632],
        [0.84949786],
        [0.19660595],
        [0.85118139]])

In [16]:
FF = np.reshape(F,(16,1),order='F')

In [17]:
np.reshape(FF,(4,4),order='F')

matrix([[0.63218146, 0.69127342, 0.18292136, 0.44698632],
        [0.85628559, 0.99533619, 0.05287674, 0.84949786],
        [0.32981306, 0.46227985, 0.71755215, 0.19660595],
        [0.98939182, 0.10466464, 0.75474602, 0.85118139]])

In [18]:
np.reshape(F,(-1,1),order='F')

matrix([[0.63218146],
        [0.85628559],
        [0.32981306],
        [0.98939182],
        [0.69127342],
        [0.99533619],
        [0.46227985],
        [0.10466464],
        [0.18292136],
        [0.05287674],
        [0.71755215],
        [0.75474602],
        [0.44698632],
        [0.84949786],
        [0.19660595],
        [0.85118139]])

In [19]:
np.matrix(np.zeros((0,1)))

matrix([], shape=(0, 1), dtype=float64)

In [20]:
np.concatenate((np.matrix(np.zeros((0,1))),FF),axis=0)

matrix([[0.63218146],
        [0.85628559],
        [0.32981306],
        [0.98939182],
        [0.69127342],
        [0.99533619],
        [0.46227985],
        [0.10466464],
        [0.18292136],
        [0.05287674],
        [0.71755215],
        [0.75474602],
        [0.44698632],
        [0.84949786],
        [0.19660595],
        [0.85118139]])

In [21]:
parnames = list(ibm_model.parameters.__dict__.keys())

In [22]:
parnames

['size', 'mu', 'F', 'Q', 'R', 'beta00', 'P00']

In [23]:
parnames.remove('size')

In [24]:
parnames

['mu', 'F', 'Q', 'R', 'beta00', 'P00']

In [25]:
parnames.sort()

In [26]:
parnames

['F', 'P00', 'Q', 'R', 'beta00', 'mu']

In [ ]:
p = np.matrix(np.zeros((0,1)))
for aname in parnames:
    mtrx = ibm_model.parameters.aname
    newmtrx = np.reshape(mtrx,(-1,1),order='F')
    p = np.concatenate((p,newmtrx),axis=0)

In [27]:
getattr(ibm_model.parameters,'F')

matrix([[0.63218146, 0.69127342, 0.18292136, 0.44698632],
        [0.85628559, 0.99533619, 0.05287674, 0.84949786],
        [0.32981306, 0.46227985, 0.71755215, 0.19660595],
        [0.98939182, 0.10466464, 0.75474602, 0.85118139]])

In [28]:
whatevernameyouwant='F'

In [29]:
getattr(ibm_model.parameters,whatevernameyouwant)

matrix([[0.63218146, 0.69127342, 0.18292136, 0.44698632],
        [0.85628559, 0.99533619, 0.05287674, 0.84949786],
        [0.32981306, 0.46227985, 0.71755215, 0.19660595],
        [0.98939182, 0.10466464, 0.75474602, 0.85118139]])

In [32]:
p = np.matrix(np.zeros((0,1)))
for aname in parnames:
    mtrx = getattr(ibm_model.parameters,aname)
    newmtrx = np.reshape(mtrx,(-1,1),order='F')
    p = np.concatenate((p,newmtrx),axis=0)